In [19]:
import pandas as pd
import numpy as np
import datetime
import matplotlib as plt

In [20]:
filename="180809_EGC-HD-160"
#filename="Clearing_DGSL_MB51_180615"

In [36]:
def MB51Praser(filename, category) : 
    parts={}
    table={}    
    partnum=None
    with open(filename,"r", encoding="ISO-8859-1") as file:
        for i in range(3) : #skip the header
                line=file.readline()
                #print(line)    
        #print("===============skipped header===================")
        
        for line in file:                                                 
            
            if(line.count("|")==2) : # start part            
                splitLine = line.replace("|","").split()
                #print(splitLine[0])
                if (len(splitLine[0])>5) : #partnum line
                    if partnum!=None : 
                        parts[partnum] = table
                        partnum=splitLine[0]
                        table=parts.get(partnum, {})                                               
                    partnum=splitLine[0]                                
                    

                else : #if document line
                    mvt=line[6:9]                   
                    month=line[line.find("/")-4:line.find("/")+3]
                    try : 
                        qty=splitLine[splitLine.index('PC')-1]                    
                    except :
                        continue
                    
                    
                    if(qty.find("-")>0) : 
                        qty=-int(float(qty.replace("-","")))
                    else : 
                        qty=int(float(qty))              
                    
                    #print(partnum, mvt,qty)
                    
                    if(category=="qty") : 
                        table[mvt] = table.get(mvt,0) + qty                                                
                        #print(partnum, mvt, table[mvt])     
                    if(category=="count") : 
                        table[mvt] = table.get(mvt,0) + 1                                               
                        #print(partnum, mvt, table[mvt])     
                    
                    if(category=="consumption") and (mvt=='601' or mvt=='261') : 
                        table[month] = table.get(month,0) + qty                                               
                    
                    
                    if(category=="krconsumption") and (mvt=='601' or mvt=='261') :                         
                        client=line[95:103].strip()
                        if client=='642504' :                             
                            table[month] = table.get(month,0) + qty                                               
                    
                    if(category=="incoming") and mvt=='101' : 
                        table[month] = table.get(month,0) + qty                                               
                        #print(partnum, month, table[month])     
                    
                    if(category=="customer") and (mvt=='601' or mvt=='261') :                         
                        client=line[95:103].strip()                                      
                        table[client] = table.get(client,0) - qty                                        
                    
    if partnum!=None and len(table)!=0: 
        parts[partnum] = table                
    return parts

In [37]:
parts1=MB51Praser(filename,'count')
parts2=MB51Praser(filename,'qty')
parts3=MB51Praser(filename,'consumption')
parts4=MB51Praser(filename,'incoming')
parts5=MB51Praser(filename,'krconsumption')
parts6=MB51Praser(filename,'customer')

In [38]:
df1=pd.DataFrame(parts1).transpose()
df2=pd.DataFrame(parts2).transpose()
df3=pd.DataFrame(parts3).transpose()
df4=pd.DataFrame(parts4).transpose()
df5=pd.DataFrame(parts5).transpose()
df6=pd.DataFrame(parts6)



In [39]:
writer = pd.ExcelWriter(filename + '.xlsx')
df1.to_excel(writer,'Count')
df2.to_excel(writer,'Qty')
df3.to_excel(writer,'Consumption')
df4.to_excel(writer,'Incoming')
df5.to_excel(writer,'krConsumption')
df6.to_excel(writer,'customer')
writer.save()